In [8]:
# install packages
!pip install pytesseract
!pip install pdf2image

You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [9]:
# import packages
import pytesseract
from pdf2image import convert_from_path
import os
import re
from datetime import datetime
import pandas as pd

In [10]:
# Function to extract text from a PDF file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

def extract_text_from_pdf(pdf_path):
    # Convert the PDF to a list of PIL Image objects
    images = convert_from_path(pdf_path)
    text_list = []
    # Loop through each page of the PDF and extract text using Tesseract
    for image in images:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    # Combine text from all pages into a single string
    text = '\n'.join(text_list)
    return text

In [11]:
# Define the regex pattern to match the date

title_pattern = r"1\.\s+\w+(\s+\w+)*"

In [12]:
date_pattern = r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
# Extract year (A) and date (J)
def extract_date(text):
    date_match = re.search(date_pattern, text)
    if date_match:
        date = date_match.group(0)
        date_obj = datetime.strptime(date, '%d %B %Y')
        # change mm/dd/yyyy format
        new_date = date_obj.strftime('%m/%d/%Y') #column J
        year = new_date[-4:] #column A
    else:
        year = date = 'N/A'
        #print("No date found")
    return year, date

In [37]:
def remove_empty(text):
    if text == None:
        return 'N/A'
    text = text.replace('\n', ' ')
    text = text.strip()
    if text == '':
        return 'N/A'
    else:
        return text
    
    
council_pattern1 = r"(?<=session).*?(?=Agenda)"
council_pattern2 = r'([A-Z][A-Za-z]+\s)+(COUNCIL|Council)'

# Extract council/committe name (B)
def extract_council(text):
    match1 = re.search(council_pattern1, text, re.DOTALL)
    match2 = re.search(council_pattern2, text, re.DOTALL)
    council = 'N/A'
    # handle the council after session
    if match1:
        council = match1.group(0).strip()
    # handle the council before session
    if match2:
        council = match2.group(0).strip()
    if 'Economic and Social' in text:
        council = 'Economic and Social Council'
    return remove_empty(council)

In [14]:
session_pattern = r'(?P<session>[\w-]+\ssession)'
# Extract session (C)
def extract_session(text):
    match = re.search(session_pattern, text)
    if match:
        session = match.group("session")
    else:
        session = 'N/A'
        print("Session information not found.")
    return session

In [90]:
agenda_pattern = r'Agenda item (\d+)(?: \((\w+)\))?(?:\n\n)?'
draft_pattern = r':? *([\w ]*|[\n]*)? ?draft' #': *([\w ]*)?draft'

# Extract Agenda item (D), Agenda detail (E), and countries (F)
def extract_agenda_countries(text):
    match1 = re.search(agenda_pattern, text)
    if match1:
        agenda_item_number = match1.group(1)
        agenda_item_letter = match1.group(2)
        if agenda_item_letter is not None:
            agenda_item = f"{agenda_item_number} ({agenda_item_letter})"
        else:
            agenda_item = agenda_item_number
        
        start_index = match1.end()

        match2 = re.search(draft_pattern, text[start_index:], re.DOTALL)
        if match2:
            end_index = start_index + match2.start()
            content = text[start_index:end_index]
            parts = content.split("\n\n")
            parts = [remove_empty(s) for s in parts]
            agenda_detail = parts[0]
            countries = ''
            for part in parts[1:]:
                if part[-2] == ':':
                    countries += part[:-2]
                else:
                    countries += part
        else:
            print('match2 error')
    
    return agenda_item, remove_empty(agenda_detail), remove_empty(countries)

In [16]:
def split_text(text):
    # delete extracted text
    split_text = re.split(draft_pattern, text, flags=re.IGNORECASE)
    #print(split_text)
    #print(len(split_text))
    part1_text = split_text[0]
    part2_text = split_text[2]
    return part1_text, part2_text

In [59]:
text_head_pattern = r'(The General Assembly,|The Commission on Human Rights,|The Human Rights Council |The Economic and Social Council,|1\.)'
number_title_pattern = r'^\s*(\d{4})?/?\W*(.*)'

# Extract body title number (G) and body title （H）
def extract_body_title(text):
    title_body_text = re.split(text_head_pattern, text)
    body = ' '.join(title_body_text[1:])
    title_text = title_body_text[0]
    title_text = title_text.replace('\n', ' ')
    
    match1 = re.search(r'[A-Z\d]', title_text)
    if match1:
        index = match1.start()
        title_text = title_text[index-1:]
        match2 = re.match(number_title_pattern, title_text)
        if match2:
            title_number = match2.group(1)
            title = match2.group(2)
    else:
        title_number = 'N/A'
        title = 'N/A'

    title = remove_empty(title)
    if len(title.split(' ')) < 3:
        title = 'N/A'

    return remove_empty(title_number), title, body

In [79]:
footnote_pattern = r'^(\*|\d(?![./])|\d{1}/)\s.*'

# Extract body text (I) and footnote (M)
def extract_body(text):
    # extract the text content from the second non-empty paragraph
    body_text = ''
    footnote = []
    footnote_idx = 1
    paragraphs = text.split('\n\n')
    #print(paragraphs)
    
    for paragraph in paragraphs:
        #for line in paragraph.split('\n'):
            # exclude lines containing specific patterns
        paragraph = paragraph.replace("\n", " ")
        #print(paragraph)
        match = re.search(footnote_pattern, paragraph)
        if match:
            if match.group(0)[0] == '*':
                footnote.append(match.group(0))
            elif match.group(0)[0] == str(footnote_idx):
                footnote.append(match.group(0))
                footnote_idx += 1
        else:
            # if len(paragraph) > 50 or paragraph == "The General Assembly,":
            #     body_text += paragraph.strip() + ' '
            # else:
            if len(paragraph) == 0:
                continue
            if (paragraph[0] == str(footnote_idx) and paragraph[1] == '/') or paragraph[0] == "*":
                footnote_idx += 1
                footnote.append(paragraph) 
            if not re.search(r'(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle|Page \d{1})', paragraph):
                body_text += paragraph.strip() + ' '
    #print(footnote)
    footnote = ' '.join(footnote)
    return body_text, footnote

In [86]:
# Function to classify text into column A to N
def classify_text_to_column(text):
    result = []
    
    # Split based on : draft, which is the end of countries
    part1_text, part2_text = split_text(text)
    
    # Get year(A) and date(J)
    year, date = extract_date(part1_text)

    # Get council/committe name (B)
    council = extract_council(part1_text)

    # Get session (C)
    session = extract_session(part1_text)

    # Get agenda item (D), agend detail (E), and countries (F)
    agenda_item, agenda_detail, countries = extract_agenda_countries(text)

    # Get title number (G) and title body (H)
    title_number, title_text, body = extract_body_title(part2_text)

    # Get Body text (I)
    body_text, footnote = extract_body(body)
    
    # Add into result list
    result = [year, council, session, agenda_item, agenda_detail, countries, title_number, title_text, body_text, date, footnote]
    
    return result

In [85]:
# Path to the folder containing the PDF files
pdf_folder = './Dataset'

# Path to the folder where the CSV files will be saved
csv_folder = './csv_folder'

# csv file column list
column_list = ['year', 'Council', 'Session', 'Agenda item', 'Agenda detail', 'cosponsored countries', 
               'body title number',	'body title detail', 'body text', 'date', 'file', 'filecountry', 'footnote', 'scanned']

In [98]:
# Loop through each folder and PDF file and extract text
for country_folder in os.listdir(pdf_folder):
    print('---------------' + country_folder + '---------------' )
    if not os.path.isdir(os.path.join(pdf_folder, country_folder)):
        continue
    csv_path = os.path.join(csv_folder, f"{country_folder}.csv")
    rows = []
    count = 0
    # Loop through each file 
    csv_year = 1994
    for pdf_file in os.listdir(os.path.join(pdf_folder, country_folder)):
        if not pdf_file.endswith('.pdf'):
            continue
        # check if this file is old-version
        year = int(pdf_file.split('_')[0])
        if year < 1994:
            continue

        pdf_path = os.path.join(pdf_folder, country_folder, pdf_file).replace('\\', '/')
        print(pdf_path)
        # try:
        text = extract_text_from_pdf(pdf_path)
        text_columns = classify_text_to_column(text)
        row = text_columns[:-1] + [pdf_file[:-4], country_folder] + text_columns[-1:] #add column K and L
        
        # check if year was extracted correctlys
        if row[0] != year:
            row[0] = year
        
        # scanned (N), cannot know 
        if int(year) > 1990:
            scanned = 'yes'
        else:
            scanned = 'no'
        row.append(scanned)

        rows.append(row)
        count += 1
        #print(rows)
        if year == 1995:
            df = pd.DataFrame(rows, columns = column_list)
            df.to_csv(csv_path, index = False) 
            break
        # except:
        #     print(f"Error extracting text from {pdf_path}")
        #     continue
    break
    # Write the rows to the CSV file
    df = pd.DataFrame(rows, columns = column_list)
    df.to_csv(csv_path, index = False)  

---------------Afghanistan---------------
./Dataset/Afghanistan/1994_163336.pdf
./Dataset/Afghanistan/1994_163532.pdf
./Dataset/Afghanistan/1994_163619.pdf
./Dataset/Afghanistan/1994_163690.pdf
./Dataset/Afghanistan/1994_163780.pdf
./Dataset/Afghanistan/1994_163787.pdf
./Dataset/Afghanistan/1994_163866.pdf
./Dataset/Afghanistan/1994_163874.pdf
./Dataset/Afghanistan/1994_163890.pdf
./Dataset/Afghanistan/1994_163906.pdf
./Dataset/Afghanistan/1994_163951.pdf
./Dataset/Afghanistan/1994_163971.pdf
./Dataset/Afghanistan/1994_164403.pdf
./Dataset/Afghanistan/1994_164949.pdf
./Dataset/Afghanistan/1994_164984.pdf
./Dataset/Afghanistan/1994_165025.pdf
./Dataset/Afghanistan/1994_165103.pdf
./Dataset/Afghanistan/1994_165106.pdf
./Dataset/Afghanistan/1994_165169.pdf
./Dataset/Afghanistan/1994_165194.pdf
./Dataset/Afghanistan/1994_165216.pdf
./Dataset/Afghanistan/1994_165219.pdf
./Dataset/Afghanistan/1994_165966.pdf
./Dataset/Afghanistan/1994_166182.pdf
./Dataset/Afghanistan/1994_167388.pdf
./Datase

### TEST

In [100]:
test_list = ['1994_163532.pdf', '1994_163690.pdf', '1994_163787.pdf', '1994_163866.pdf', '1994_172588.pdf', '1999_395501.pdf', '2000_412394.pdf', '2006_584361.pdf'] 
rows = []
for test_pdf in test_list:
    if test_pdf == '1994_172588.pdf':
        test_text = extract_text_from_pdf(f'./Dataset/Afghanistan/{test_pdf}')
        #print(test_text)
        #break
        #p1, p2 = split_text(test_text)
        #print(p1)
        #print(p2)
        # test year and date
        #print(extract_date(p1))
        #print(extract_council(p1))
        #print(extract_session(p1))
        #print(extract_agenda_countries(test_text))
        #remain = split_text(test_text)
        #t_n, t, b = extract_body_title(p2)
        #print(extract_body(b))

    #rt = re.split(title, p2)
    #print(len(rt))
        year = int(test_pdf.split('_')[0])
        text_columns = classify_text_to_column(test_text)
        row = text_columns[:-1] + [test_pdf[:-4], 'country_folder'] + text_columns[-1:] 
    
        if row[0] != year:
            row[0] = year

        # scanned (N), cannot know 
        if int(year) > 1990:
            scanned = 'yes'
        else:
            scanned = 'no'
        row.append(scanned)
        print(row)
        break
    rows.append(row)

    #print(rows)
    #df = pd.DataFrame(rows, columns = column_list)

[1994, 'N/A', 'Forty-ninth session', '156', 'BUILDING A PEACEFUL AND BETTER WORLD THROUGH SPORT', "Algeria, Angola, Antigua and Barbuda, Armenia, Australia, Bahamas, Barbados, Belarus, Benin, Bolivia, Botswana, Bulgaria, Burkina Faso, Burundi, Cameroon, Cape Verde, Comoros, Congo, Céte d'Ivoire, Cyprus, Czech Republic, Djibouti, Dominican Republic, Egypt, Eritrea, Ethiopia, Gabon, Gambia, Georgia, Ghana, Greece, Guinea, Guinea-Bissau, Guyana, Haiti, Honduras, Hungary, Iceland, India, Indonesia, Israel, Jamaica, Japan, Kazakhstan, Kenya, Lesotho, Liberia, Libyan Arab Jamahiriya, Liechtenstein, Luxembourg, Madagascar, Malawi, Mali, Malta, Mauritania, Mauritius, Monaco, Mongolia, Mozambique, Namibia, New Zealand, Niger, Nigeria, Norway, Pakistan, Panama, Papua New Guinea, Republic of Korea, Rwanda, Saint Kitts and Nevis, Saint Vincent and_ the Grenadines, Samoa, San Marino, Senegal, Seychelles, Sierra Leone, Slovakia, South Africa, Spain, Sudan, Suriname, Swaziland, the former Yugoslav Re

In [93]:
df

,year,Council,Session,Agenda item,Agenda detail,cosponsored countries,body title number,body title detail,body text,date,file,filecountry,footnote,scanned
0,1994,SPECIAL POLITICAL AND DECOLONIZATION COMMITTEE...,Forty-ninth session,74,PROTECTION AND SECURITY OF SMALL STATES,"Afghanistan, Antigua and Barbuda, Australia, B...",N/A,N/A,"The General Assembly, Recalling its resolution...",18 October 1994,1994_163532,country_folder,"1/ Resolution 2625 (XXV), annex. 2/ B/49/353.",yes
1,1994,N/A,Forty-ninth session,22,ASSISTANCE IN MINE CLEARANCE,"Afghanistan, Bolivia, Colombia, Costa Rica, El...",N/A,N/A,1. Insert a new operative paragraph 8 reading...,27 October 1994,1994_163690,country_folder,,yes
2,1994,N/A,Forty-ninth session,22,ASSISTANCE IN MINE CLEARANCE,Afghanistan,N/A,N/A,1. At the end of operative paragraph 7 insert...,25 October 1994,1994_163787,country_folder,,yes
3,1994,THIRD COMMITTEE,Forty-ninth session,95,"SOCIAL DEVELOPMENT, INCLUDING QUESTIONS RELATI...","Afghanistan, Armenia, Bangladesh, Belarus, Bel...",N/A,Towards full integration of persons with disab...,"The General Assembly, Recalling its resolution...",1 November 1994,1994_163866,country_folder,"1/ A/37/351/Add.1 and Add.1/Corr.1, annex, sec...",yes
4,1999,N/A,Fifty-fourth session,20 (b),Strengthening of the coordination of humanitar...,"Afghanistan, Armenia, Azerbaijan, Bangladesh, ...",N/A,"Emergency international assistance for peace, ...","The General Assembly, Recalling its resolution...",N/A,1999_395501,country_folder,,yes
5,2000,Economic and Social Council,Fifty-sixth session,14 (b),SPECIFIC GROUPS AND INDIVIDUALS MINORITIES,"Afghanistan*, Albania*, Algeria*, Armenia*, Au...",2000,Tolerance and pluralism as indivisible element...,"The Commission on Human Rights, Recalling the ...",18 April 2000,2000_412394,country_folder,"* Tn accordance with rule 69, paragraph 3, of ...",yes
6,2006,HUMAN RIGHTS COUNCIL,Second session,2,IMPLEMENTATION OF GENERAL ASSEMBLY RESOLUTION ...,Finland (on behalf of the European Union) and ...,2006,Cooperation with the Office of the United Nati...,The Human Rights Council decides to adopt the...,3 October 2006,2006_584361,country_folder,* Non-member States of the Human Rights Council.,yes
